# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1e3834c550>
├── 'a' --> tensor([[-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722]])
└── 'x' --> <FastTreeValue 0x7f1e3834c970>
    └── 'c' --> tensor([[ 1.0751,  0.5819,  0.2164, -0.5943],
                        [-1.7324, -0.0305, -0.0945,  0.8597],
                        [ 0.4221,  1.5757, -0.8090,  0.3270]])

In [4]:
t.a

tensor([[-0.2411, -0.2262, -0.2681],
        [ 0.2169,  0.4751, -0.9722]])

In [5]:
%timeit t.a

58.9 ns ± 0.0469 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1e3834c550>
├── 'a' --> tensor([[ 1.0964, -0.0338, -0.4916],
│                   [ 2.0080,  0.0396,  0.3370]])
└── 'x' --> <FastTreeValue 0x7f1e3834c970>
    └── 'c' --> tensor([[ 1.0751,  0.5819,  0.2164, -0.5943],
                        [-1.7324, -0.0305, -0.0945,  0.8597],
                        [ 0.4221,  1.5757, -0.8090,  0.3270]])

In [7]:
%timeit t.a = new_value

57.9 ns ± 0.0404 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722]]),
    x: Batch(
           c: tensor([[ 1.0751,  0.5819,  0.2164, -0.5943],
                      [-1.7324, -0.0305, -0.0945,  0.8597],
                      [ 0.4221,  1.5757, -0.8090,  0.3270]]),
       ),
)

In [10]:
b.a

tensor([[-0.2411, -0.2262, -0.2681],
        [ 0.2169,  0.4751, -0.9722]])

In [11]:
%timeit b.a

54.8 ns ± 0.0525 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5526, -0.1049,  0.1275],
               [ 0.1695,  0.4744,  0.1065]]),
    x: Batch(
           c: tensor([[ 1.0751,  0.5819,  0.2164, -0.5943],
                      [-1.7324, -0.0305, -0.0945,  0.8597],
                      [ 0.4221,  1.5757, -0.8090,  0.3270]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 50.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 313 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1e382d8970>
├── 'a' --> tensor([[[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]],
│           
│                   [[-0.2411, -0.2262, -0.2681],
│                    [ 0.2169,  0.4751, -0.9722]]])
└── 'x' --> <FastTreeValue 0x7f1e382d8820>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 68.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1d8cf78e50>
├── 'a' --> tensor([[-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722],
│                   [-0.2411, -0.2262, -0.2681],
│                   [ 0.2169,  0.4751, -0.9722]])
└── 'x' --> <FastTreeValue 0x7f1d8cfe1fd0>
    └── 'c' --> tensor([[ 1.0751,  0.5819,  0.2164, -0.5943],
                        [-1.7324, -0.0305, -0.0945,  0.8597],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 79.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 329 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]],
       
               [[-0.2411, -0.2262, -0.2681],
                [ 0.2169,  0.4751, -0.9722]]]),
    x: Batch(
           c: tensor([[[ 1.0751,  0.5819,  0.2164, -0.5943],
                       [-1.7324, -0.0305, -0.0945,  0.8597],
                       [ 0.4221,  1.5757, -0.8090,  0.3270]],
         

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722],
               [-0.2411, -0.2262, -0.2681],
               [ 0.2169,  0.4751, -0.9722]]),
    x: Batch(
           c: tensor([[ 1.0751,  0.5819,  0.2164, -0.5943],
                      [-1.7324, -0.0305, -0.0945,  0.8597],
                      [ 0.4221,  1.5757, -0.8090,  0.3270],
                      [ 1.0751,  0.5819,  0.2164, -0.5943],
                      [-1.7324, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 500 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 772 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
